# SNPE flexible 

In [1]:
import torch

from sbi.inference import SNPE, prepare_for_sbi, simulate_for_sbi
from sbi.utils.get_nn_models import posterior_nn
from sbi import utils as utils
from sbi import analysis as analysis

Define simulator and prior


In [3]:
import numpy as np
from tabulate import tabulate
import matplotlib.pyplot as plt
from scipy.integrate import odeint

# Define the model
def model(variables, t, params):
    m1, p1, m2, p2, m3, p3 = variables
    k1, k2 = params

    dm1dt = -m1 + (10 ** 3 / (1 + (10 ** k1 * p2) ** 2)) + 1
    dp1dt = -10 ** 0 * (p1 - m1)

    dm2dt = -m2 + (10 ** 3 / (1 + (10 ** k2 * p3) ** 2)) + 1
    dp2dt = -10 ** 0 * (p2 - m2)

    dm3dt = -m3 + (10 ** 3 / (1 + (10 ** 0 * p1) ** 2)) + 1
    dp3dt = -10 ** 0 * (p3 - m3)

    return [dm1dt, dp1dt, dm2dt, dp2dt, dm3dt, dp3dt]

#Define true parameters
true_params = np.array([
    0, 0  # first set of odes
])

def ode_solver(parameter_set):
    initial_conditions = np.array([0, 1, 0, 3, 0, 2])
    t = np.linspace(0, 100, 1000) #num_timesteps is 1000
    solution = odeint(model, initial_conditions, t, args=(parameter_set,))
    return solution

true_data = ode_solver(true_params) #True trajectories

In [4]:
num_dim = 2
prior = utils.BoxUniform(low=-3 * torch.ones(num_dim), high=3 * torch.ones(num_dim))

Start with SNPE

In [5]:
simulator, prior = prepare_for_sbi(ode_solver, prior)

/project/home23/sg2023/Desktop/SBI/.conda/lib/python3.11/site-packages/scipy/integrate/_odepack_py.py:242: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  output = _odepack.odeint(func, y0, t, args, Dfun, col_deriv, ml, mu,


In [6]:
#Instantiate the inference object:
inference = SNPE(prior=prior)

In [8]:
theta, x = simulate_for_sbi(simulator, proposal=prior, num_simulations=500)

Running 500 simulations.:   0%|          | 0/500 [00:00<?, ?it/s]

In [9]:
inference = inference.append_simulations(theta, x)

In [10]:
inference

In [11]:
x_o = torch.tensor(true_data)
x_o


tensor([[  0.0000,   1.0000,   0.0000,   3.0000,   0.0000,   2.0000],
        [  8.9123,   1.3648,  12.9051,   3.4811,  42.9083,   4.0501],
        [ 13.8703,   2.3506,  14.4150,   4.4862,  61.6269,   8.8289],
        ...,
        [ 46.9959, 113.7526,  56.8882,  25.9989,   1.2610,   2.7314],
        [ 42.7398, 107.1846,  63.3755,  29.2513,   1.2439,   2.5905],
        [ 38.8630, 100.8554,  70.3635,  32.8362,   1.2295,   2.4615]],
       dtype=torch.float64)

In [14]:
density_estimator = inference.train()
posterior = inference.build_posterior(density_estimator)

Neural network successfully converged after 108 epochs.


In [1]:
posterior

NameError: name 'posterior' is not defined

In [15]:
posterior_samples = posterior.sample((10000,), x=x_o)

ValueError: The `x` passed to condition the posterior for evaluation or sampling
                has an inferred batch shape larger than one. This is not supported in
                some sbi methods for reasons depending on the scenario:

                    - in case you want to evaluate or sample conditioned on several xs
                    e.g., (p(theta | [x1, x2, x3])), this is not supported yet except
                    when using likelihood based SNLE and SNRE.

                    - in case you trained with a single round to do amortized inference
                    and now you want to evaluate or sample a given theta conditioned on
                    several xs, one after the other, e.g, p(theta | x1), p(theta | x2),
                    p(theta| x3): this broadcasting across xs is not supported in sbi.
                    Instead, what you can do it to call posterior.log_prob(theta, xi)
                    multiple times with different xi.

                    - finally, 